In [1]:
import tensorflow as tf
import pathlib
import imageio
#tf.enable_eager_execution()
import os
import pandas as pd




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#path=os.getcwd()+'/drive/My Drive/Colab Notebooks/sp-society-camera-model-identification'
#print(path)
path = '/content/drive/My Drive/Colab Notebooks/sp-society-camera-model-identification'
os.chdir(path)

In [4]:
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


In [5]:
trainpath = path+"/train"
os.chdir(trainpath)

df1 = pd.read_pickle('train_images1.pkl')
df1.head(5)


,location,image
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float..."
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3..."
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float..."
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float..."
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3..."


In [6]:
df1.shape

(1375, 2)

In [7]:
df2 = pd.read_pickle('train_images2.pkl')
df2.shape

(1375, 2)

In [8]:

df1 = pd.concat([df1,df2])
df1.shape

(2750, 2)

In [9]:
df1['camera'] = df1['location'].apply(lambda x: x.split('\\')[-2])
df1.head()    

,location,image,camera
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float...",HTC-1-M7
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3...",HTC-1-M7
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float...",HTC-1-M7
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float...",HTC-1-M7
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3...",HTC-1-M7


In [10]:

txt= '''cameras = os.listdir()
cameras.remove('.gitignore')
cameras.remove('train_images1.pkl')
cameras.remove('train_images2.pkl')
cameras.remove('model_weights.h5')
filecount = [len(os.listdir(camera))  for camera in cameras]
cam = pd.DataFrame({'camera_type':cameras,'filecount':filecount})

class_dict = {}
for id in cam.index:
    class_dict.update({cam.loc[id,'camera_type']:id})
'''    
class_dict = {'HTC-1-M7': 0,
 'iPhone-4s': 1,
 'iPhone-6': 2,
 'LG-Nexus-5x': 3,
 'Motorola-Droid-Maxx': 4,
 'Motorola-Nexus-6': 5,
 'Motorola-X': 6,
 'Samsung-Galaxy-Note3': 7,
 'Samsung-Galaxy-S4': 8,
 'Sony-NEX-7': 9}

In [ ]:
df1['class'] = df1['camera'].apply(lambda x:class_dict[x])
df1.head()

In [ ]:
df1['image_array'] = df1['image'].apply(lambda x:x.numpy())

In [ ]:
#getting training and test datasets
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(df1['image_array'], df1['class'].values,test_size=.33, random_state=42)


In [ ]:
del df1

In [ ]:


import numpy as np
def reshape_input(Xtrain):
    #Xtrain = np.stack(Xtrain.values) #convert a pandas series of arrays into one single array
    input_cols = Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3]
    Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],input_cols)) #flatten the data to single set of neurons
    return Xtrain

In [ ]:
def reshape_input_cnn(Xtrain):
    Xtrain = np.stack(Xtrain.values) #convert a pandas series of arrays into one single array
    return Xtrain

In [ ]:
#categorize output data. 
import keras
from keras.utils.np_utils import to_categorical

def reshape_target(Ytrain):
    Ytrain = to_categorical(Ytrain,num_classes=10)
    return Ytrain

#Ytrain = reshape_target(Ytrain)

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam


In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
Xtrain.shape

In [ ]:
def get_model(Xtrain,Ytrain,output_units=500):
    model = Sequential()
    input_cols = Xtrain.shape[1]
    model.add(Dense(output_units, activation='relu',input_shape=(input_cols,)))
    model.add(Dense(output_units))
    model.add(Dense(output_units))
    model.add(Dense(output_units))
    #model.add(Dense(output_units))
    #model.add(Dense(output_units))
    model.add(Dense(10,activation='softmax'))
    return model

In [ ]:
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
def get_cnnmodel(X,Y):
  model=Sequential()
  
  shape = Xtrain[0].shape
  model.add(Conv2D(64, kernel_size=3, activation='relu',input_shape=shape))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, kernel_size=3, activation='relu'))
  model.add(Flatten())
  model.add(Dense(10, activation='softmax'))
  return model

In [ ]:
#this function is used to modify the generator's output to suit NN's need instead of a CNN
def datagen_modifier(X, Y,batchsize):
  datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)
  while True:
    for batch in datagen.flow(np.stack(X.values),Y,batch_size= batchsize):
      batch_x = reshape_input(batch[0])
      batch_y = reshape_target(batch[1])
      yield (batch_x,batch_y)

In [ ]:
Xtrain = reshape_input_cnn(Xtrain)
Ytrain = reshape_target(Ytrain)
xtrain, xval, ytrain, yval = train_test_split(Xtrain, Ytrain,test_size=.2, random_state=42)

In [ ]:
#Testing a normal neural network
from keras.preprocessing.image import ImageDataGenerator
earlystopping_monitor = EarlyStopping(monitor='val_accuracy',min_delta=.1,patience=10,verbose=1)
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)


#lr = [0.0001]
#changing the lr from .001 to .0001 to increase accuracy more than 32


#create validation split for the fit_generator
  

#datagen.fit() no need to fit as the featurewise_center is set to false.
trainingsize = 6400 #keep the number as a multiple of batchsize to get same number of images each time
batchsize = 64
iteration = 1
itersize = trainingsize//batchsize
# first_run_flag = True
# print(itersize)
# for batch in datagen.flow(np.stack(Xtrain.values),Ytrain,batch_size= batchsize):
#     batch_x = reshape_input(batch[0])
#     batch_y = reshape_target(batch[1])
#     print(batch_x.shape)
#     print(batch_y.shape)
#     if first_run_flag:
#       print('This happens once')
#       model = get_model(batch_x,batch_y,2000)
#       #model.load_weights('model_weights.h5')
#       optimizer = SGD(0.001)#default learning rate is .001
#       model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
#       first_run_flag=False
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='model_weights_cnn_v2.h5',
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)
batch_x = np.full((64,192*192*3),1)
batch_y = np.full((64,10),1)
model = get_cnnmodel(batch_x,batch_y)
optimizer = Adam(0.001)#default learning rate is .001
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
#model.load_weights('model_weights_cnn.h5')
model.fit_generator(datagen.flow(xtrain,ytrain,batchsize), steps_per_epoch=5120, epochs=3, 
                    validation_data=datagen.flow(xval,yval,batchsize),validation_steps= 1280,verbose=1,callbacks=[model_checkpoint_callback])
#model.fit(xtrain,ytrain,validation_split=.3,epochs=40,callbacks=[earlystopping_monitor])
print('next batch')
    # iteration+=1
    # if(iteration >itersize ):
    #     break


#model.save_weights('model_weights.h5')
#using lr of .001 has the best improvement in accuracy so far(.17 to .48 over 10 epoches) So, going lesser than that and increaseing the epoches to 25
#.00001 is slow as it increases accuracy from.10 to .27 in 25 epoches
#.0001 .13->.46 in 25 epoches (caused resouorceexhaustedError) 
#.001 .17 -> .72 (it fluctuates after .71, maxes at .72 and decreases again). 
#Increasing the output units to 1000 to see the increase, with lr of .001. Accuracy reaches upto .76 but its not converging. 
#WOULD REDUCING LEARNING RATE AFTER 20 EPOCHES HELP TO CONVEREGE?
#validation accuracy is still at .28 max

#to use Model checkpoint, refer link here: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
#preprocessing and training
#training->hyperparameter
#preprocessing -> rotate imags and others

In [ ]:
model.save('cnn_model.h5')
model.summary()